In [2]:
import sys
sys.path.append("../")

In [3]:
from utils.text_splitter import TextSplitter
from utils.chroma_store import ChromaStore
from chromadb.utils import embedding_functions
from transformers import pipeline
import pandas as pd
import time
import uuid
import matplotlib.pyplot as plt
from IPython.display import display

pd.set_option('max_colwidth', 800)

Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
NumExpr defaulting to 8 threads.


In [4]:
### Parameters
data_version = "data/first_version"
USE_FULL_DATASET = False
DEFAULT_EMBED_INSTRUCTION = "Represent the document for retrieval: "
DEFAULT_QUERY_INSTRUCTION = (
    "Represent the question for retrieving supporting documents: "
)

chroma_client = ChromaStore(chroma_server_hostname="localhost", chroma_server_port=8000)

EMBED_MODEL = "hkunlp/instructor-base"
LLM_MODEL = "google/flan-t5-base"
N_RESULTS = 4

QUESTIONS = [
"Can you tell me about depression?",
"What lifestyle changes are suggested for depression?",
"Are there different types of depressions?"
]

TEMPLATE = "Question: {question}\n\nContext: {context}\n\n"

MAX_OUTPUT_LENGTH = 200
TEMPERATURE = 0.1

Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.


In [5]:
nhs_fixed_context = """Overview - Depression in adults Depression is more than simply feeling unhappy or fed up for a few days. Most people go through periods of feeling down, but when you're depressed you feel persistently sad for weeks or months, rather than just a few days. Some people think depression is trivial and not a genuine health condition. They're wrong – it is a real illness with real symptoms. Depression is not a sign of weakness or something you can "snap out of" by "pulling yourself together". The good news is that with the right treatment and support, most people with depression can make a full recovery. These pages are about depression in adults. Read about depression in children and young people .

How to tell if you have depression Depression affects people in different ways and can cause a wide variety of symptoms. They range from lasting feelings of unhappiness and hopelessness, to losing interest in the things you used to enjoy and feeling very tearful. Many people with depression also have symptoms of anxiety . There can be physical symptoms too, such as feeling constantly tired, sleeping badly, having no appetite or sex drive, and various aches and pains. The symptoms of depression range from mild to severe. At its mildest, you may simply feel persistently low in spirit, while severe depression can make you feel suicidal, that life is no longer worth living. Most people experience feelings of stress, anxiety or low mood during difficult times. A low mood may improve after a short period of time, rather than being a sign of depression.

When to see a doctor It's important to seek help from a GP if you think you may be depressed. Many people wait a long time before seeking help for depression, but it's best not to delay. The sooner you see a doctor, the sooner you can be on the way to recovery.

What causes depression? Sometimes there's a trigger for depression. Life-changing events, such as bereavement, losing your job or giving birth, can bring it on. People with a family history of depression are more likely to experience it themselves. But you can also become depressed for no obvious reason.

Treating depression Treatment for depression can involve a combination of lifestyle changes, talking therapies and medicine. Your recommended treatment will be based on how severe your depression is. If you have mild depression, your doctor may suggest waiting to see whether it improves on its own, while monitoring your progress. This is known as "watchful waiting". They may also suggest lifestyle measures such as exercise and guided self-help. Talking therapies, such as cognitive behavioural therapy (CBT) , may also be used for mild depression. For moderate to severe depression, a combination of talking therapy and antidepressants is often recommended. If you have severe depression, you may be referred to a specialist mental health team for intensive specialist talking treatments and prescribed medicine.

Living with depression Many people with depression benefit by making lifestyle changes, such as getting more exercise , cutting down on alcohol , giving up smoking and eating healthily . Reading a self-help book or joining a support group are also worthwhile. They can help you gain a better understanding about what causes you to feel depressed. Sharing your experiences with others in a similar situation can also be very supportive.

Information: Social care and support guide If you:
need help with day-to-day living because of illness or
disability care for someone regularly because they're ill or disabled, or because of their age – including family members
Our social care and support guide explains your options and where you can get support.

Symptoms - Depression in adults The symptoms of depression can be complex and vary widely between people. If you're depressed, you may feel sad, hopeless and lose interest in things you used to enjoy. The symptoms persist for weeks or months and are bad enough to interfere with your work, social life and family life. There are many other symptoms of depression and you're unlikely to have all of those listed on this page.

Psychological symptoms The psychological symptoms of depression include: continuous low mood or sadness feeling hopeless and helpless having low self-esteem feeling tearful feeling guilt-ridden feeling irritable and intolerant of others having no motivation or interest in things finding it difficult to make decisions not getting any enjoyment out of life feeling anxious or worried having suicidal thoughts or thoughts of harming yourself

Physical symptoms The physical symptoms of depression include: moving or speaking more slowly than usual changes in appetite or weight (usually decreased, but sometimes increased) constipation unexplained aches and pains lack of energy low sex drive ( loss of libido ) disturbed sleep – for example, finding it difficult to fall asleep at night or waking up very early in the morning

Social symptoms The social symptoms of depression include: avoiding contact with friends and taking part in fewer social activities neglecting your hobbies and interests having difficulties in your home, work or family life

Severities of depression Depression can often come on gradually, so it can be difficult to notice something is wrong. Many people try to cope with their symptoms without realising they're unwell. It can sometimes take a friend or family member to suggest something is wrong. Doctors describe depression in adults as either less severe (mild) or more severe (moderate or severe), based on: the symptoms, including how often you get symptoms and how bad they are how long depression lasts the impact on your daily life A few people with severe depression may have symptoms of psychosis . You'll usually be asked to answer a set of questions to help doctors assess how severe your condition is.

Grief and depression It can be difficult to distinguish between grief and depression. They share many of the same characteristics, but there are important differences between them. Grief is an entirely natural response to a loss, while depression is an illness. People who are grieving may have symptoms of depression, but they usually improve with time. For some people, bereavement can lead to depression. If you're grieving it's normal to have feelings of sadness. Some other symptoms of depression are rare with grief and bereavement, such as having suicidal thoughts, symptoms of psychosis and feeling hopeless or guilt-ridden.

Other types of depression There are different types of depression, and some conditions where depression may be one of the symptoms. These include: postnatal depression – sometimes new mothers, fathers or partners develop depression after they have a baby; this is known as postnatal depression and it's treated in a similar way to other types of depression, with talking therapies and antidepressant medicines bipolar disorder – also known as "manic depression", in bipolar disorder there are spells of both depression and excessively high mood (mania); the depression symptoms are similar to depression, but the bouts of mania can include harmful behaviour, such as gambling, going on spending sprees and having unsafe sex seasonal affective disorder (SAD) – also known as "winter depression", SAD is a type of depression with a seasonal pattern usually related to winter premenstrual dysphoric disorder (PMDD) – a severe type of premenstrual syndrome (PMS) where you have symptoms including feelings of depression and anxiety in the weeks before your period

When to seek help See a GP if you experience symptoms of depression for most of the day, every day, for more than 2 weeks. A low mood may improve after a short time.
"""

mind_fixed_context = """Depression
Learn about depression, its symptoms and possible causes, and how you can access treatment and support. Find tips on caring for yourself, and guidance for friends and family.

What is depression?
Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you feel or behave.

The symptoms you experience may vary. How intense they are, how long they last, and how much they affect your daily life can also vary.

If you experience milder depression, you might have low mood but still be able to carry on with your daily life. But things may feel harder and less worthwhile.

If you have more severe depression, you might find day-to-day life much more difficult. You may also experience suicidal feelings.

It starts as sadness then I feel myself shutting down, becoming less capable of coping. Eventually, I just feel numb and empty.

When does low mood become depression?
We all have times when our mood is low, and we feel sad or fed up. Often these feelings happen for a reason and pass on their own.

But it might be depression if the feelings become so bad that they interfere with our daily life. Or if they last for several weeks or months.

Are there different types of depression?
If you're diagnosed with depression, you might also be told that it is ‘less severe’ or ‘more severe’. This describes how your symptoms are affecting you, and what treatment you're likely to be offered. You may find that the severity of your depression changes over time.

Sometimes you might hear depression being called ‘major depressive disorder’. There are some other types of depression too:

Persistent depressive disorder (PDD). PDD is continuous depression that lasts for two years or more. You may also hear it called dysthymia or chronic depression.
Seasonal affective disorder (SAD). SAD is depression that occurs at a particular time of year, or during a particular season. See our page on SAD for more information.
Antenatal depression. This is depression that occurs while you are pregnant. It is sometimes called prenatal depression.
Postnatal depression (PND). This is depression that occurs in the first year after having a baby. This can include affecting dads and partners.
Premenstrual dysphoric disorder (PMDD). This is a hormone-related disorder that affects your body but also how you feel. This can involve experiencing depression. So your doctor may describe this as a mental health problem.


Common signs and symptoms of depression
These are some common signs of depression that you may experience:

How you might feel
Down, upset or tearful
Restless, agitated or irritable
Guilty, worthless and down on yourself
Empty and numb
Isolated and unable to relate to other people
Finding no pleasure in life or things you usually enjoy
Angry or frustrated over minor things
A sense of unreality
No self-confidence or self-esteem
Hopeless and despairing
Feeling tired all the time
How you might act
Avoiding social events and activities you usually enjoy
Self-harming or suicidal behaviour
Difficulty speaking, thinking clearly or making decisions
Losing interest in sex
Difficulty remembering or concentrating on things
Using more tobacco, alcohol or other drugs than usual
Difficulty sleeping, or sleeping too much
No appetite and losing weight, or eating more than usual and gaining weight
Physical aches and pains with no obvious physical cause
Moving very slowly, or being restless and agitated
It felt like I was really tired, all the time. I had no energy or emotion about anything.

Anxiety
It's very common to experience depression and anxiety together. Some symptoms of depression can also be symptoms of anxiety, for example:

Feeling restless
Finding it difficult to concentrate
Struggling to sleep

I flit between states of anxiety and depression. At times, each seems to fuel the other.

Self-harm and suicidal feelings
If you're feeling low, you might self-harm to cope with difficult feelings. Although this might make you feel better in the short term, self-harm can be very dangerous.

When you're feeling really low and hopeless, you may also find yourself thinking about suicide. This could be thinking about the idea of suicide, or considering a plan to end your life. These thoughts can feel difficult to control, and can be very frightening.

Psychotic symptoms
If you experience depression, you might also experience some psychotic symptoms. These may include delusions, such as paranoia. Or they may be hallucinations, such as hearing voices.

If you experience psychotic symptoms as part of depression, they're likely to be linked to your depressed thoughts and feelings. This may include experiencing delusions relating to feelings of guilt. For example, you might become convinced that you've committed a crime.

These kinds of experiences can feel very real at the time. This might make it hard to understand that these experiences are also symptoms of your depression. And they can be frightening or upsetting, so it's important to seek treatment and support.

You might worry that experiencing psychotic symptoms could mean you’re given a diagnosis that doesn’t feel right for you. But discussing all of your symptoms with your doctor can help you get the right support and treatment.

How might depression affect my day-to-day life?
If you have depression, you might find that it interferes with different aspects of your life. For example, it might affect your ability to work, your relationships, or managing your finances. This can add extra stress to an already difficult experience.

It may also feel hard to explain your thoughts and feelings to others. You might want to cut yourself off from other people instead. This could lead to you feeling isolated and lonely.

It feels like I'm stuck under a huge grey-black cloud. It's dark and isolating, smothering me at every opportunity.

Can depression be a symptom of other mental health problems?
Symptoms of depression can also be part of other mental health problems, such as:

Bipolar disorder
Borderline personality disorder (BPD) and other personality disorders
Schizoaffective disorder

If you experience low mood or suicidal thoughts, this might be the reason you first speak to your doctor about your mental health. And your doctor might offer you treatment for depression without realising that you also experience other symptoms.

If you think you're experiencing other symptoms, you can talk to your doctor about this to make sure you're getting the right treatment.
"""


nhs_row = [
['2576149a-d0a0-48bb-a7b4-03da36697912', """Overview - Depression in adults Depression is more than simply feeling unhappy or fed up for a few days. Most people go through periods of feeling down, but when you're depressed you feel persistently sad for weeks or months, rather than just a few days. Some people think depression is trivial and not a genuine health condition. They're wrong – it is a real illness with real symptoms. Depression is not a sign of weakness or something you can "snap out of" by "pulling yourself together". The good news is that with the right treatment and support, most people with depression can make a full recovery. These pages are about depression in adults. Read about depression in children and young people. How to tell if you have depression Depression affects people in different ways and can cause a wide variety of symptoms. They range from lasting feelings of unhappiness and hopelessness, to losing interest in the things you used to enjoy and feeling very tearful. Many people with depression also have symptoms of anxiety . There can be physical symptoms too, such as feeling constantly tired, sleeping badly, having no appetite or sex drive, and various aches and pains. The symptoms of depression range from mild to severe. At its mildest, you may simply feel persistently low in spirit, while severe depression can make you feel suicidal, that life is no longer worth living. Most people experience feelings of stress, anxiety or low mood during difficult times. A low mood may improve after a short period of time, rather than being a sign of depression. When to see a doctor It's important to seek help from a GP if you think you may be depressed. Many people wait a long time before seeking help for depression, but it's best not to delay. The sooner you see a doctor, the sooner you can be on the way to recovery. What causes depression? Sometimes there's a trigger for depression. Life-changing events, such as bereavement, losing your job or giving birth, can bring it on. People with a family history of depression are more likely to experience it themselves. But you can also become depressed for no obvious reason. Treating depression Treatment for depression can involve a combination of lifestyle changes, talking therapies and medicine. Your recommended treatment will be based on how severe your depression is. If you have mild depression, your doctor may suggest waiting to see whether it improves on its own, while monitoring your progress. This is known as "watchful waiting". They may also suggest lifestyle measures such as exercise and guided self-help. Talking therapies, such as cognitive behavioural therapy (CBT) , may also be used for mild depression. For moderate to severe depression, a combination of talking therapy and antidepressants is often recommended. If you have severe depression, you may be referred to a specialist mental health team for intensive specialist talking treatments and prescribed medicine. Living with depression Many people with depression benefit by making lifestyle changes, such as getting more exercise , cutting down on alcohol , giving up smoking and eating healthily . Reading a self-help book or joining a support group are also worthwhile. They can help you gain a better understanding about what causes you to feel depressed. Sharing your experiences with others in a similar situation can also be very supportive. Information: Social care and support guide If you: need help with day-to-day living because of illness or disability care for someone regularly because they're ill or disabled, or because of their age – including family members Our social care and support guide explains your options and where you can get support.""", '2023-07-20 14:42:38.491223', 'https://www.nhs.uk/mental-health/conditions/depression-in-adults/causes/'],
['2576149a-d0a0-48bb-a7b4-03da36697931', """Symptoms - Depression in adults The symptoms of depression can be complex and vary widely between people. If you're depressed, you may feel sad, hopeless and lose interest in things you used to enjoy. The symptoms persist for weeks or months and are bad enough to interfere with your work, social life and family life. There are many other symptoms of depression and you're unlikely to have all of those listed on this page. Psychological symptoms The psychological symptoms of depression include: continuous low mood or sadness feeling hopeless and helpless having low self-esteem feeling tearful feeling guilt-ridden feeling irritable and intolerant of others having no motivation or interest in things finding it difficult to make decisions not getting any enjoyment out of life feeling anxious or worried having suicidal thoughts or thoughts of harming yourself. Physical symptoms The physical symptoms of depression include: moving or speaking more slowly than usual changes in appetite or weight (usually decreased, but sometimes increased) constipation unexplained aches and pains lack of energy low sex drive ( loss of libido ) disturbed sleep – for example, finding it difficult to fall asleep at night or waking up very early in the morning. Social symptoms The social symptoms of depression include: avoiding contact with friends and taking part in fewer social activities neglecting your hobbies and interests having difficulties in your home, work or family life. Severities of depression Depression can often come on gradually, so it can be difficult to notice something is wrong. Many people try to cope with their symptoms without realising they're unwell. It can sometimes take a friend or family member to suggest something is wrong. Doctors describe depression in adults as either less severe (mild) or more severe (moderate or severe), based on: the symptoms, including how often you get symptoms and how bad they are how long depression lasts the impact on your daily life A few people with severe depression may have symptoms of psychosis . You'll usually be asked to answer a set of questions to help doctors assess how severe your condition is. Grief and depression It can be difficult to distinguish between grief and depression. They share many of the same characteristics, but there are important differences between them. Grief is an entirely natural response to a loss, while depression is an illness. People who are grieving may have symptoms of depression, but they usually improve with time. For some people, bereavement can lead to depression. If you're grieving it's normal to have feelings of sadness. Some other symptoms of depression are rare with grief and bereavement, such as having suicidal thoughts, symptoms of psychosis and feeling hopeless or guilt-ridden. Other types of depression There are different types of depression, and some conditions where depression may be one of the symptoms. These include: postnatal depression – sometimes new mothers, fathers or partners develop depression after they have a baby; this is known as postnatal depression and it's treated in a similar way to other types of depression, with talking therapies and antidepressant medicines bipolar disorder – also known as "manic depression", in bipolar disorder there are spells of both depression and excessively high mood (mania); the depression symptoms are similar to depression, but the bouts of mania can include harmful behaviour, such as gambling, going on spending sprees and having unsafe sex seasonal affective disorder (SAD) – also known as "winter depression", SAD is a type of depression with a seasonal pattern usually related to winter premenstrual dysphoric disorder (PMDD) – a severe type of premenstrual syndrome (PMS) where you have symptoms including feelings of depression and anxiety in the weeks before your period. When to seek help See a GP if you experience symptoms of depression for most of the day, every day, for more than 2 weeks. A low mood may improve after a short time.""", '2023-07-20 14:42:38.491223','https://www.nhs.uk/mental-health/conditions/depression-in-adults/overview/']
]


mind_row = [
['2576149a-d0a0-48bb-a7b4-03da36697900', """Depression Learn about depression, its symptoms and possible causes, and how you can access treatment and support. Find tips on caring for yourself, and guidance for friends and family.  What is depression? Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you feel or behave.  The symptoms you experience may vary. How intense they are, how long they last, and how much they affect your daily life can also vary.  If you experience milder depression, you might have low mood but still be able to carry on with your daily life. But things may feel harder and less worthwhile.  If you have more severe depression, you might find day-to-day life much more difficult. You may also experience suicidal feelings.  It starts as sadness then I feel myself shutting down, becoming less capable of coping. Eventually, I just feel numb and empty.  When does low mood become depression? We all have times when our mood is low, and we feel sad or fed up. Often these feelings happen for a reason and pass on their own.  But it might be depression if the feelings become so bad that they interfere with our daily life. Or if they last for several weeks or months.  Are there different types of depression? If you're diagnosed with depression, you might also be told that it is ‘less severe’ or ‘more severe’. This describes how your symptoms are affecting you, and what treatment you're likely to be offered. You may find that the severity of your depression changes over time.  Sometimes you might hear depression being called ‘major depressive disorder’. There are some other types of depression too:  Persistent depressive disorder (PDD). PDD is continuous depression that lasts for two years or more. You may also hear it called dysthymia or chronic depression. Seasonal affective disorder (SAD). SAD is depression that occurs at a particular time of year, or during a particular season. See our page on SAD for more information. Antenatal depression. This is depression that occurs while you are pregnant. It is sometimes called prenatal depression. Postnatal depression (PND). This is depression that occurs in the first year after having a baby. This can include affecting dads and partners. Premenstrual dysphoric disorder (PMDD). This is a hormone-related disorder that affects your body but also how you feel. This can involve experiencing depression. So your doctor may describe this as a mental health problem.""", '2023-07-21 14:42:38.491223', 'https://www.mind.org.uk/information-support/types-of-mental-health-problems/depression/about-depression/'],
['2576149a-d0a0-48bb-a7b4-03da36697901', """Common signs and symptoms of depression These are some common signs of depression that you may experience:  How you might feel Down, upset or tearful Restless, agitated or irritable Guilty, worthless and down on yourself Empty and numb Isolated and unable to relate to other people Finding no pleasure in life or things you usually enjoy Angry or frustrated over minor things A sense of unreality No self-confidence or self-esteem Hopeless and despairing Feeling tired all the time How you might act Avoiding social events and activities you usually enjoy Self-harming or suicidal behaviour Difficulty speaking, thinking clearly or making decisions Losing interest in sex Difficulty remembering or concentrating on things Using more tobacco, alcohol or other drugs than usual Difficulty sleeping, or sleeping too much No appetite and losing weight, or eating more than usual and gaining weight Physical aches and pains with no obvious physical cause Moving very slowly, or being restless and agitated It felt like I was really tired, all the time. I had no energy or emotion about anything. Anxiety It's very common to experience depression and anxiety together. Some symptoms of depression can also be symptoms of anxiety, for example: Feeling restless, Finding it difficult to concentrate, Struggling to sleep I flit between states of anxiety and depression. At times, each seems to fuel the other. Self-harm and suicidal feelings If you're feeling low, you might self-harm to cope with difficult feelings. Although this might make you feel better in the short term, self-harm can be very dangerous. When you're feeling really low and hopeless, you may also find yourself thinking about suicide. This could be thinking about the idea of suicide, or considering a plan to end your life. These thoughts can feel difficult to control, and can be very frightening. Psychotic symptoms If you experience depression, you might also experience some psychotic symptoms. These may include delusions, such as paranoia. Or they may be hallucinations, such as hearing voices. If you experience psychotic symptoms as part of depression, they're likely to be linked to your depressed thoughts and feelings. This may include experiencing delusions relating to feelings of guilt. For example, you might become convinced that you've committed a crime. These kinds of experiences can feel very real at the time. This might make it hard to understand that these experiences are also symptoms of your depression. And they can be frightening or upsetting, so it's important to seek treatment and support. You might worry that experiencing psychotic symptoms could mean you’re given a diagnosis that doesn’t feel right for you. But discussing all of your symptoms with your doctor can help you get the right support and treatment. How might depression affect my day-to-day life? If you have depression, you might find that it interferes with different aspects of your life. For example, it might affect your ability to work, your relationships, or managing your finances. This can add extra stress to an already difficult experience. It may also feel hard to explain your thoughts and feelings to others. You might want to cut yourself off from other people instead. This could lead to you feeling isolated and lonely. It feels like I'm stuck under a huge grey-black cloud. It's dark and isolating, smothering me at every opportunity. Can depression be a symptom of other mental health problems? Symptoms of depression can also be part of other mental health problems, such as: Bipolar disorder, Borderline personality disorder (BPD) and other personality disorders, Schizoaffective disorder. If you experience low mood or suicidal thoughts, this might be the reason you first speak to your doctor about your mental health. And your doctor might offer you treatment for depression without realising that you also experience other symptoms. If you think you're experiencing other symptoms, you can talk to your doctor about this to make sure you're getting the right treatment.""", '2023-07-20 14:42:38.491223' ,'https://www.mind.org.uk/information-support/types-of-mental-health-problems/depression/symptoms/']
]


In [6]:
if USE_FULL_DATASET:
    nhs_data = pd.read_csv('../data/nhs_data_validated.csv')
    mind_data = pd.read_csv('../data/mind_data_validated.csv')
else:
    nhs_data =  pd.DataFrame(nhs_row, columns=['uuid', 'text_scraped', 'timestamp', 'url'])
    mind_data =  pd.DataFrame(mind_row, columns=['uuid', 'text_scraped', 'timestamp', 'url'])
    
print(f"Length of NHS dataset: {len(nhs_data)}")
print(f"Length of Mind dataset: {len(mind_data)}")

Length of NHS dataset: 2
Length of Mind dataset: 2


In [30]:
def split_using_chunking(df, chunk_size, chunk_overlap):
    texts = df["text_scraped"].values.tolist()
    src_urls = df["url"].values.tolist()

    print(f"Using chunk_size={chunk_size} and chunk_overlap={chunk_overlap}")
    text_splitter = TextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

    chunks, metadatas = [], []
    for text, url in zip(texts, src_urls):
        split_chunks = text_splitter.split_text(text)
        chunks.extend(split_chunks)
        metadatas.extend(
            [
                {"source": url, "data_version": data_version}
                for _ in range(len(split_chunks))
            ]
        )

    uuids = [str(uuid.uuid4()) for _ in range(len(chunks))]
    return uuids, chunks, metadatas

def add_chroma(chunk_size, chunk_overlap, enable_chunking):
    ef_embed = embedding_functions.InstructorEmbeddingFunction(model_name=EMBED_MODEL, instruction=DEFAULT_EMBED_INSTRUCTION)

    start = time.perf_counter()
    for df, collection_name in zip([nhs_data, mind_data], ['nhs_collection', 'mind_collection']):
        print(f"Inserting data into collection {collection_name}")
        print(f"Chunking enabled = {enable_chunking}")
        
        if enable_chunking:
            uuids, chunks, metadatas = split_using_chunking(df, chunk_size, chunk_overlap)
        
        else:
            uuids = df["uuid"].values.tolist()
            chunks = df["text_scraped"].values.tolist()
            src_urls = df["url"].values.tolist()
            metadatas = [{"source": url, "data_version": data_version} for url in src_urls]
            
        chroma_client.add_texts(
            collection_name=collection_name,
            texts=chunks,
            ids=uuids,
            metadatas=metadatas,
            embedding_function=ef_embed,
        )    
    print(f"Time taken to insert data in chroma store {time.perf_counter()-start} sec")


def query_chroma(collection_name, query_text):
    ef_query = embedding_functions.InstructorEmbeddingFunction(model_name=EMBED_MODEL, instruction=DEFAULT_QUERY_INSTRUCTION)

    result_dict = chroma_client.query_collection(
        collection_name=collection_name,
        query_texts=query_text,
        n_results=N_RESULTS,
        embedding_function=ef_query,
    )
    documents = " ".join(result_dict["documents"][0])
    return documents


def get_llm_prompt_response(questions, enable_chunking, is_new_experiment, fixed_context=False, chunk_size = 1000, chunk_overlap = 200):
    
    collection_names = chroma_client.list_collection_names()
    # Delete any existing data from chroma store
    if is_new_experiment:
        print(f"Found {len(collection_names)} collections")
        for collection_name in collection_names:
            chroma_client.delete_collection(collection_name)
        
        add_chroma(chunk_size, chunk_overlap, enable_chunking)

    collection_names = sorted(chroma_client.list_collection_names())

    results = []
    for query_text in questions:
        responses = []
        for collection_name in collection_names:
            print(f"Collection name: {collection_name}")
    
            if fixed_context:
                context = mind_fixed_context if 'mind' in collection_name else nhs_fixed_context
                print("Using fixed context for LLM prompt")
            else:
                context = query_chroma(collection_name, query_text)
        
            prompt = TEMPLATE.format(question=query_text, context=context)
        
            print("="*100)
            print("LLM prompt")
            print(prompt)
        
            llm = pipeline(model=LLM_MODEL, task="text2text-generation")
            data = llm(prompt, max_length=MAX_OUTPUT_LENGTH, temperature=TEMPERATURE)
            response = data[0]["generated_text"]
        
            print("="*100)
            print("LLM Response")
            print(response)
            print("="*100)

            responses.append(response)
        results.append(responses)

    results_df = pd.DataFrame(results, columns=['mind_response', 'nhs_response'])
    results_df['question'] = questions
    return results_df

## Distribution for lengths

Run the following cells only if `USE_FULL_DATASET` is set to `True`.

### NHS dataset

Average length of text in NHS dataset is 3751 words. Considering all texts are of the same length, we add these texts in vector store and get 4 nearest neighbours for query question. This puts length of context = 15004 words (3751 x 4).

In [ ]:
nhs_data['text_scraped'].apply(len).hist(grid=False)
plt.suptitle("Histogram for length distribution NHS data")

In [ ]:
nhs_data['text_scraped'].str.len().describe()

### Mind dataset

Average length of text in Mind dataset is 4965 words. Considering all texts are of the same length, we add these texts in vector store and get 4 nearest neighbours for query question. This puts length of context = 19860 words (4965 x 4).

In [ ]:
mind_data['text_scraped'].apply(len).hist(grid=False)
plt.suptitle("Histogram for length distribution Mind data")

In [ ]:
mind_data['text_scraped'].str.len().describe()

## Experiments

> Note: This requires a local chroma store running in the background. To run a chroma store server, run the following steps

```bash
docker pull ghcr.io/chroma-core/chroma:0.4.3
docker run -it -p 8000:8000 ghcr.io/chroma-core/chroma:0.4.3
```

This will start chroma server on localhost at port 8000.

### No chunking using fixed context (Baseline)

Baseline results using fixed handcrafted context.

In [38]:
results_df = get_llm_prompt_response(QUESTIONS, enable_chunking = False, is_new_experiment = True, fixed_context = True)

Found 2 collections
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
Inserting data into collection nhs_collection
Chunking enabled = False
Inserting data into collection mind_collection
Chunking enabled = False
Time taken to insert data in chroma store 1.7204358110029716 sec
Collection name: mind_collection
Using fixed context for LLM prompt
LLM prompt
Question: Can you tell me about depression?

Context: Depression
Learn about depression, its symptoms and possible causes, and how you can access treatment and support. Find tips on caring for yourself, and guidance for friends and family.

What is depression?
Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you feel or behave.

The symptoms you experience may vary. How intense they are, how long they last, and how much they affect your daily life can also vary.

If

Token indices sequence length is longer than the specified maximum sequence length for this model (1427 > 512). Running this sequence through the model will result in indexing errors


LLM Response
It's a mental health problem that involves having a low mood or losing interest and enjoyment in things.
Collection name: nhs_collection
Using fixed context for LLM prompt
LLM prompt
Question: Can you tell me about depression?

Context: Overview - Depression in adults Depression is more than simply feeling unhappy or fed up for a few days. Most people go through periods of feeling down, but when you're depressed you feel persistently sad for weeks or months, rather than just a few days. Some people think depression is trivial and not a genuine health condition. They're wrong – it is a real illness with real symptoms. Depression is not a sign of weakness or something you can "snap out of" by "pulling yourself together". The good news is that with the right treatment and support, most people with depression can make a full recovery. These pages are about depression in adults. Read about depression in children and young people .

How to tell if you have depression Depression 

Token indices sequence length is longer than the specified maximum sequence length for this model (1631 > 512). Running this sequence through the model will result in indexing errors


LLM Response
Depression is more than simply feeling unhappy or fed up for a few days. Most people go through periods of feeling down, but when you're depressed you feel persistently sad for weeks or months, rather than just a few days.
Collection name: mind_collection
Using fixed context for LLM prompt
LLM prompt
Question: What lifestyle changes are suggested for depression?

Context: Depression
Learn about depression, its symptoms and possible causes, and how you can access treatment and support. Find tips on caring for yourself, and guidance for friends and family.

What is depression?
Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you feel or behave.

The symptoms you experience may vary. How intense they are, how long they last, and how much they affect your daily life can also vary.

If you experience milder depression, you might have low mood but still be able to c

...

LLM Response
getting more exercise and cutting down on alcohol
Collection name: mind_collection
Using fixed context for LLM prompt
LLM prompt
Question: Are there different types of depressions?

Context: Depression
Learn about depression, its symptoms and possible causes, and how you can access treatment and support. Find tips on caring for yourself, and guidance for friends and family.

What is depression?
Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you feel or behave.

The symptoms you experience may vary. How intense they are, how long they last, and how much they affect your daily life can also vary.

If you experience milder depression, you might have low mood but still be able to carry on with your daily life. But things may feel harder and less worthwhile.

If you have more severe depression, you might find day-to-day life much more difficult. You may also expe

Token indices sequence length is longer than the specified maximum sequence length for this model (1428 > 512). Running this sequence through the model will result in indexing errors


LLM Response
There are some other types of depression too.
Collection name: nhs_collection
Using fixed context for LLM prompt
LLM prompt
Question: Are there different types of depressions?

Context: Overview - Depression in adults Depression is more than simply feeling unhappy or fed up for a few days. Most people go through periods of feeling down, but when you're depressed you feel persistently sad for weeks or months, rather than just a few days. Some people think depression is trivial and not a genuine health condition. They're wrong – it is a real illness with real symptoms. Depression is not a sign of weakness or something you can "snap out of" by "pulling yourself together". The good news is that with the right treatment and support, most people with depression can make a full recovery. These pages are about depression in adults. Read about depression in children and young people .

How to tell if you have depression Depression affects people in different ways and can cause a wi

Token indices sequence length is longer than the specified maximum sequence length for this model (1632 > 512). Running this sequence through the model will result in indexing errors


LLM Response
There are different types of depression, and some conditions where depression may be one of the symptoms. These include: postnatal depression – sometimes new mothers, fathers or partners develop depression after they have a baby; this is known as postnatal depression and it's treated in a similar way to other types of depression, with talking therapies and antidepressant medicines


In [41]:
results_df

,mind_response,nhs_response,question
0,It's a mental health problem that involves having a low mood or losing interest and enjoyment in things.,"Depression is more than simply feeling unhappy or fed up for a few days. Most people go through periods of feeling down, but when you're depressed you feel persistently sad for weeks or months, rather than just a few days.",Can you tell me about depression?
1,Reduce your tobacco use and alcohol intake,getting more exercise and cutting down on alcohol,What lifestyle changes are suggested for depression?
2,There are some other types of depression too.,"There are different types of depression, and some conditions where depression may be one of the symptoms. These include: postnatal depression – sometimes new mothers, fathers or partners develop depression after they have a baby; this is known as postnatal depression and it's treated in a similar way to other types of depression, with talking therapies and antidepressant medicines",Are there different types of depressions?


### Chunking

Question: Does chunking improve results?

### chunk_size = 2000 and chunk_overlap = 50

In [42]:
results_df = get_llm_prompt_response(QUESTIONS, enable_chunking = True, is_new_experiment = True, chunk_size = 2000, chunk_overlap = 50)

Found 2 collections
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
Inserting data into collection nhs_collection
Chunking enabled = True
Using chunk_size=2000 and chunk_overlap=50
Inserting data into collection mind_collection
Chunking enabled = True
Using chunk_size=2000 and chunk_overlap=50
Time taken to insert data in chroma store 3.578440422999847 sec
Collection name: mind_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: Can you tell me about depression?

Context: Depression Learn about depression, its symptoms and possible causes, and how you can access treatment and support. Find tips on caring for yourself, and guidance for friends and family.  What is depression? Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you 

Token indices sequence length is longer than the specified maximum sequence length for this model (1433 > 512). Running this sequence through the model will result in indexing errors


LLM Response
It can affect your daily life.
Collection name: nhs_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: Can you tell me about depression?

Context: Overview - Depression in adults Depression is more than simply feeling unhappy or fed up for a few days. Most people go through periods of feeling down, but when you're depressed you feel persistently sad for weeks or months, rather than just a few days. Some people think depression is trivial and not a genuine health condition. They're wrong – it is a real illness with real symptoms. Depression is not a sign of weakness or something you can "snap out of" by "pulling yourself together". The good news is that with the right treatment and support, most people with depression can make a full recovery. These pages are about depression in adults. Read about depression in children and young people. How to tell if you have depression Depression aff

Token indices sequence length is longer than the specified maximum sequence length for this model (1324 > 512). Running this sequence through the model will result in indexing errors


LLM Response
Depression is more than simply feeling unhappy or fed up for a few days. Most people go through periods of feeling down, but when you're depressed you feel persistently sad for weeks or months, rather than just a few days.
Collection name: mind_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: What lifestyle changes are suggested for depression?

Context: Depression Learn about depression, its symptoms and possible causes, and how you can access treatment and support. Find tips on caring for yourself, and guidance for friends and family.  What is depression? Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you feel or behave.  The symptoms you experience may vary. How intense they are, how long they last, and how much they affect your daily life can also vary.  If you exp

...

LLM Response
getting more exercise and cutting down on alcohol
Collection name: mind_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: Are there different types of depressions?

Context: Depression Learn about depression, its symptoms and possible causes, and how you can access treatment and support. Find tips on caring for yourself, and guidance for friends and family.  What is depression? Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you feel or behave.  The symptoms you experience may vary. How intense they are, how long they last, and how much they affect your daily life can also vary.  If you experience milder depression, you might have low mood but still be able to carry on with your daily life. But things may feel harder and less worthwhile.  If you have more severe depressi

Token indices sequence length is longer than the specified maximum sequence length for this model (1434 > 512). Running this sequence through the model will result in indexing errors


LLM Response
Persistent depressive disorder (PDD).
Collection name: nhs_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: Are there different types of depressions?

Context: lasts the impact on your daily life A few people with severe depression may have symptoms of psychosis . You'll usually be asked to answer a set of questions to help doctors assess how severe your condition is. Grief and depression It can be difficult to distinguish between grief and depression. They share many of the same characteristics, but there are important differences between them. Grief is an entirely natural response to a loss, while depression is an illness. People who are grieving may have symptoms of depression, but they usually improve with time. For some people, bereavement can lead to depression. If you're grieving it's normal to have feelings of sadness. Some other symptoms of depression are rare with grief and

Token indices sequence length is longer than the specified maximum sequence length for this model (1325 > 512). Running this sequence through the model will result in indexing errors


LLM Response
There are different types of depression, and some conditions where depression may be one of the symptoms. These include: postnatal depression – sometimes new mothers, fathers or partners develop depression after they have a baby; this is known as postnatal depression and it's treated in a similar way to other types of depression, with talking therapies and antidepressant medicines bipolar disorder – also known as "manic depression", in bipolar disorder there are spells of both depression and excessively high mood (mania); the depression symptoms are similar to depression, but the bouts of mania can include harmful behaviour, such as gambling, going on spending sprees and having unsafe sex seasonal affective disorder (SAD) – also known as "winter depression", SAD is a severe type of premenstrual syndrome (PMS) where you have symptoms including feelings of stress, anxiety or low mood during difficult times.


In [43]:
results_df

,mind_response,nhs_response,question
0,It can affect your daily life.,"Depression is more than simply feeling unhappy or fed up for a few days. Most people go through periods of feeling down, but when you're depressed you feel persistently sad for weeks or months, rather than just a few days.",Can you tell me about depression?
1,Reduce your tobacco use,getting more exercise and cutting down on alcohol,What lifestyle changes are suggested for depression?
2,Persistent depressive disorder (PDD).,"There are different types of depression, and some conditions where depression may be one of the symptoms. These include: postnatal depression – sometimes new mothers, fathers or partners develop depression after they have a baby; this is known as postnatal depression and it's treated in a similar way to other types of depression, with talking therapies and antidepressant medicines bipolar disorder – also known as ""manic depression"", in bipolar disorder there are spells of both depression and excessively high mood (mania); the depression symptoms are similar to depression, but the bouts of mania can include harmful behaviour, such as gambling, going on spending sprees and having unsafe sex seasonal affective disorder (SAD) – also known as ""winter depression"", SAD is a severe type of pre...",Are there different types of depressions?


### chunk_size = 1000 and chunk_overlap = 50

In [44]:
results_df = get_llm_prompt_response(QUESTIONS, enable_chunking = True, is_new_experiment = True, chunk_size = 1000, chunk_overlap = 50)

Found 2 collections
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
Inserting data into collection nhs_collection
Chunking enabled = True
Using chunk_size=1000 and chunk_overlap=50
Inserting data into collection mind_collection
Chunking enabled = True
Using chunk_size=1000 and chunk_overlap=50
Time taken to insert data in chroma store 2.9004106339998543 sec
Collection name: mind_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: Can you tell me about depression?

Context: Depression Learn about depression, its symptoms and possible causes, and how you can access treatment and support. Find tips on caring for yourself, and guidance for friends and family.  What is depression? Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you

Token indices sequence length is longer than the specified maximum sequence length for this model (697 > 512). Running this sequence through the model will result in indexing errors


LLM Response
symptoms
Collection name: nhs_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: Can you tell me about depression?

Context: Overview - Depression in adults Depression is more than simply feeling unhappy or fed up for a few days. Most people go through periods of feeling down, but when you're depressed you feel persistently sad for weeks or months, rather than just a few days. Some people think depression is trivial and not a genuine health condition. They're wrong – it is a real illness with real symptoms. Depression is not a sign of weakness or something you can "snap out of" by "pulling yourself together". The good news is that with the right treatment and support, most people with depression can make a full recovery. These pages are about depression in adults. Read about depression in children and young people. How to tell if you have depression Depression affects people in differe

Token indices sequence length is longer than the specified maximum sequence length for this model (851 > 512). Running this sequence through the model will result in indexing errors


LLM Response
depression is more than simply feeling unhappy or fed up for a few days. Most people go through periods of feeling down, but when you're depressed you feel persistently sad for weeks or months, rather than just a few days.
Collection name: mind_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: What lifestyle changes are suggested for depression?

Context: If you have depression, you might find that it interferes with different aspects of your life. For example, it might affect your ability to work, your relationships, or managing your finances. This can add extra stress to an already difficult experience. It may also feel hard to explain your thoughts and feelings to others. You might want to cut yourself off from other people instead. This could lead to you feeling isolated and lonely. It feels like I'm stuck under a huge grey-black cloud. It's dark and isolating, smothering me at ev

...

LLM Response
getting more exercise , cutting down on alcohol , giving up smoking and eating healthily
Collection name: mind_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: Are there different types of depressions?

Context: When does low mood become depression? We all have times when our mood is low, and we feel sad or fed up. Often these feelings happen for a reason and pass on their own.  But it might be depression if the feelings become so bad that they interfere with our daily life. Or if they last for several weeks or months.  Are there different types of depression? If you're diagnosed with depression, you might also be told that it is ‘less severe’ or ‘more severe’. This describes how your symptoms are affecting you, and what treatment you're likely to be offered. You may find that the severity of your depression changes over time.  Sometimes you might hear depression being called ‘major 

Token indices sequence length is longer than the specified maximum sequence length for this model (632 > 512). Running this sequence through the model will result in indexing errors


LLM Response
Persistent depressive disorder (PDD). Seasonal affective disorder (SAD). Antenatal depression (PND). Postnatal depression (PND). This is depression that occurs at a particular SAD is depression that occurs at a particular time of year, or during a particular season.
Collection name: nhs_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: Are there different types of depressions?

Context: types of depression, and some conditions where depression may be one of the symptoms. These include: postnatal depression – sometimes new mothers, fathers or partners develop depression after they have a baby; this is known as postnatal depression and it's treated in a similar way to other types of depression, with talking therapies and antidepressant medicines bipolar disorder – also known as "manic depression", in bipolar disorder there are spells of both depression and excessively high mood (mania);

Token indices sequence length is longer than the specified maximum sequence length for this model (880 > 512). Running this sequence through the model will result in indexing errors


LLM Response
There are different types of depression, and some conditions where very tearful. Many people with depression also have symptoms of anxiety


In [45]:
results_df

,mind_response,nhs_response,question
0,symptoms,"depression is more than simply feeling unhappy or fed up for a few days. Most people go through periods of feeling down, but when you're depressed you feel persistently sad for weeks or months, rather than just a few days.",Can you tell me about depression?
1,cut yourself off from other people instead,"getting more exercise , cutting down on alcohol , giving up smoking and eating healthily",What lifestyle changes are suggested for depression?
2,"Persistent depressive disorder (PDD). Seasonal affective disorder (SAD). Antenatal depression (PND). Postnatal depression (PND). This is depression that occurs at a particular SAD is depression that occurs at a particular time of year, or during a particular season.","There are different types of depression, and some conditions where very tearful. Many people with depression also have symptoms of anxiety",Are there different types of depressions?


### chunk_size = 1500 and chunk_overlap = 50

In [36]:
results_df = get_llm_prompt_response(QUESTIONS, enable_chunking = True, is_new_experiment = True, chunk_size = 1500, chunk_overlap = 50)

Found 2 collections
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
Inserting data into collection nhs_collection
Chunking enabled = True
Using chunk_size=1500 and chunk_overlap=50
Inserting data into collection mind_collection
Chunking enabled = True
Using chunk_size=1500 and chunk_overlap=50
Time taken to insert data in chroma store 2.616422502003843 sec
Collection name: mind_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: Can you tell me about depression?

Context: Depression Learn about depression, its symptoms and possible causes, and how you can access treatment and support. Find tips on caring for yourself, and guidance for friends and family.  What is depression? Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you 

Token indices sequence length is longer than the specified maximum sequence length for this model (1153 > 512). Running this sequence through the model will result in indexing errors


LLM Response
It can affect your daily life.
Collection name: nhs_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: Can you tell me about depression?

Context: Overview - Depression in adults Depression is more than simply feeling unhappy or fed up for a few days. Most people go through periods of feeling down, but when you're depressed you feel persistently sad for weeks or months, rather than just a few days. Some people think depression is trivial and not a genuine health condition. They're wrong – it is a real illness with real symptoms. Depression is not a sign of weakness or something you can "snap out of" by "pulling yourself together". The good news is that with the right treatment and support, most people with depression can make a full recovery. These pages are about depression in adults. Read about depression in children and young people. How to tell if you have depression Depression aff

Token indices sequence length is longer than the specified maximum sequence length for this model (1262 > 512). Running this sequence through the model will result in indexing errors


LLM Response
Depression is more than simply feeling unhappy or fed up for a few days. Most people go through periods of feeling down, but when you're depressed you feel persistently sad for weeks or months, rather than just a few days.
Collection name: mind_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: What lifestyle changes are suggested for depression?

Context: you, and what treatment you're likely to be offered. You may find that the severity of your depression changes over time.  Sometimes you might hear depression being called ‘major depressive disorder’. There are some other types of depression too:  Persistent depressive disorder (PDD). PDD is continuous depression that lasts for two years or more. You may also hear it called dysthymia or chronic depression. Seasonal affective disorder (SAD). SAD is depression that occurs at a particular time of year, or during a particular season. See

...

LLM Response
getting more exercise , cutting down on alcohol , giving up smoking and eating healthily
Collection name: mind_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: Are there different types of depressions?

Context: you, and what treatment you're likely to be offered. You may find that the severity of your depression changes over time.  Sometimes you might hear depression being called ‘major depressive disorder’. There are some other types of depression too:  Persistent depressive disorder (PDD). PDD is continuous depression that lasts for two years or more. You may also hear it called dysthymia or chronic depression. Seasonal affective disorder (SAD). SAD is depression that occurs at a particular time of year, or during a particular season. See our page on SAD for more information. Antenatal depression. This is depression that occurs while you are pregnant. It is sometimes called prenat

Token indices sequence length is longer than the specified maximum sequence length for this model (1154 > 512). Running this sequence through the model will result in indexing errors


LLM Response
There are some other types of depression too: Persistent depressive disorder (PDD). PDD is continuous depression that lasts for two years or more. You may also hear it called ‘Dysthymia or chronic depression.
Collection name: nhs_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: Are there different types of depressions?

Context: in your home, work or family life. Severities of depression Depression can often come on gradually, so it can be difficult to notice something is wrong. Many people try to cope with their symptoms without realising they're unwell. It can sometimes take a friend or family member to suggest something is wrong. Doctors describe depression in adults as either less severe (mild) or more severe (moderate or severe), based on: the symptoms, including how often you get symptoms and how bad they are how long depression lasts the impact on your daily life A few people 

Token indices sequence length is longer than the specified maximum sequence length for this model (1221 > 512). Running this sequence through the model will result in indexing errors


LLM Response
There are different types of depression, and some conditions where depression may be one of the symptoms. These depression may be one of the symptoms. These include: postnatal depression – sometimes new mothers, fathers or partners develop depression after they have a baby; this is known as postnatal depression and it's treated in a similar way to other types of depression, with talking therapies and antidepressant medicines bipolar disorder – also known as "manic depression", in bipolar disorder there are spells of both depression and excessively high mood (mania); the depression symptoms are similar to depression, but the bouts of mania can include harmful behaviour, such as gambling, going on spending sprees and having unsafe sex seasonal affective disorder (SAD) – also known as "winter depression", SAD is a severe type of premenstrual syndrome (PMS) where you have symptoms including feelings of depression and anxiety in


In [37]:
results_df

,mind_response,nhs_response,question
0,It can affect your daily life.,"Depression is more than simply feeling unhappy or fed up for a few days. Most people go through periods of feeling down, but when you're depressed you feel persistently sad for weeks or months, rather than just a few days.",Can you tell me about depression?
1,Cut yourself off from other people,"getting more exercise , cutting down on alcohol , giving up smoking and eating healthily",What lifestyle changes are suggested for depression?
2,There are some other types of depression too: Persistent depressive disorder (PDD). PDD is continuous depression that lasts for two years or more. You may also hear it called ‘Dysthymia or chronic depression.,"There are different types of depression, and some conditions where depression may be one of the symptoms. These depression may be one of the symptoms. These include: postnatal depression – sometimes new mothers, fathers or partners develop depression after they have a baby; this is known as postnatal depression and it's treated in a similar way to other types of depression, with talking therapies and antidepressant medicines bipolar disorder – also known as ""manic depression"", in bipolar disorder there are spells of both depression and excessively high mood (mania); the depression symptoms are similar to depression, but the bouts of mania can include harmful behaviour, such as gambling, going on spending sprees and having unsafe sex seasonal affective disorder (SAD) – also known as ""wi...",Are there different types of depressions?


### chunk_size = 1000 and chunk_overlap = 300

In [40]:
results_df = get_llm_prompt_response(QUESTIONS, enable_chunking = True, is_new_experiment = True, chunk_size = 1000, chunk_overlap = 300)

Found 2 collections
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
Inserting data into collection nhs_collection
Chunking enabled = True
Using chunk_size=1000 and chunk_overlap=300
Inserting data into collection mind_collection
Chunking enabled = True
Using chunk_size=1000 and chunk_overlap=300
Time taken to insert data in chroma store 3.0195826810013386 sec
Collection name: mind_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: Can you tell me about depression?

Context: Depression Learn about depression, its symptoms and possible causes, and how you can access treatment and support. Find tips on caring for yourself, and guidance for friends and family.  What is depression? Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how y

Token indices sequence length is longer than the specified maximum sequence length for this model (883 > 512). Running this sequence through the model will result in indexing errors


LLM Response
It starts as sadness then I feel myself shutting down, becoming less capable of coping. Eventually, I just feel numb and empty.
Collection name: nhs_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: Can you tell me about depression?

Context: Overview - Depression in adults Depression is more than simply feeling unhappy or fed up for a few days. Most people go through periods of feeling down, but when you're depressed you feel persistently sad for weeks or months, rather than just a few days. Some people think depression is trivial and not a genuine health condition. They're wrong – it is a real illness with real symptoms. Depression is not a sign of weakness or something you can "snap out of" by "pulling yourself together". The good news is that with the right treatment and support, most people with depression can make a full recovery. These pages are about depression in adults. Read

Token indices sequence length is longer than the specified maximum sequence length for this model (849 > 512). Running this sequence through the model will result in indexing errors


LLM Response
depression
Collection name: mind_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: What lifestyle changes are suggested for depression?

Context: affect my day-to-day life? If you have depression, you might find that it interferes with different aspects of your life. For example, it might affect your ability to work, your relationships, or managing your finances. This can add extra stress to an already difficult experience. It may also feel hard to explain your thoughts and feelings to others. You might want to cut yourself off from other people instead. This could lead to you feeling isolated and lonely. It feels like I'm stuck under a huge grey-black cloud. It's dark and isolating, smothering me at every opportunity. Can depression be a symptom of other mental health problems? Symptoms of depression can also be part of other mental health problems, such as: Bipolar disorder, Borderl

...

LLM Response
exercise and guided self-help
Collection name: mind_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: Are there different types of depressions?

Context: have more severe depression, you might find day-to-day life much more difficult. You may also experience suicidal feelings.  It starts as sadness then I feel myself shutting down, becoming less capable of coping. Eventually, I just feel numb and empty.  When does low mood become depression? We all have times when our mood is low, and we feel sad or fed up. Often these feelings happen for a reason and pass on their own.  But it might be depression if the feelings become so bad that they interfere with our daily life. Or if they last for several weeks or months.  Are there different types of depression? If you're diagnosed with depression, you might also be told that it is ‘less severe’ or ‘more severe’. This describes how your symptom

Token indices sequence length is longer than the specified maximum sequence length for this model (772 > 512). Running this sequence through the model will result in indexing errors


LLM Response
We all have times when our mood is low, and we feel sad or fed up. Often these feelings happen for a reason and pass on their own. But it might be depression if the feelings become so bad that they interfere with our daily life. Or if they last for several weeks or months.
Collection name: nhs_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: Are there different types of depressions?

Context: Other types of depression There are different types of depression, and some conditions where depression may be one of the symptoms. These include: postnatal depression – sometimes new mothers, fathers or partners develop depression after they have a baby; this is known as postnatal depression and it's treated in a similar way to other types of depression, with talking therapies and antidepressant medicines bipolar disorder – also known as "manic depression", in bipolar disorder there are spells 

Token indices sequence length is longer than the specified maximum sequence length for this model (856 > 512). Running this sequence through the model will result in indexing errors


LLM Response
There are different types of depression, and some conditions where depression may be one of the symptoms. These include: postnatal depression – sometimes new mothers, fathers or partners develop depression after they have a baby; this is known as postnatal depression and it's treated in a similar way to other types of depression, with talking therapies and antidepressant medicines bipolar disorder – also known as "manic depression", in bipolar disorder there are spells of both depression and excessively high mood (mania); the depression symptoms are similar to depression, but the bouts of mania can include harmful behaviour, such as gambling, going on spending sprees and having unsafe sex


In [41]:
results_df

,mind_response,nhs_response,question
0,"It starts as sadness then I feel myself shutting down, becoming less capable of coping. Eventually, I just feel numb and empty.",depression,Can you tell me about depression?
1,We all experience depression,exercise and guided self-help,What lifestyle changes are suggested for depression?
2,"We all have times when our mood is low, and we feel sad or fed up. Often these feelings happen for a reason and pass on their own. But it might be depression if the feelings become so bad that they interfere with our daily life. Or if they last for several weeks or months.","There are different types of depression, and some conditions where depression may be one of the symptoms. These include: postnatal depression – sometimes new mothers, fathers or partners develop depression after they have a baby; this is known as postnatal depression and it's treated in a similar way to other types of depression, with talking therapies and antidepressant medicines bipolar disorder – also known as ""manic depression"", in bipolar disorder there are spells of both depression and excessively high mood (mania); the depression symptoms are similar to depression, but the bouts of mania can include harmful behaviour, such as gambling, going on spending sprees and having unsafe sex",Are there different types of depressions?


### chunk_size = 1500 and chunk_overlap = 300

In [42]:
results_df = get_llm_prompt_response(QUESTIONS, enable_chunking = True, is_new_experiment = True, chunk_size = 1500, chunk_overlap = 300)

Found 2 collections
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
Inserting data into collection nhs_collection
Chunking enabled = True
Using chunk_size=1500 and chunk_overlap=300
Inserting data into collection mind_collection
Chunking enabled = True
Using chunk_size=1500 and chunk_overlap=300
Time taken to insert data in chroma store 3.2210005879969685 sec
Collection name: mind_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: Can you tell me about depression?

Context: Depression Learn about depression, its symptoms and possible causes, and how you can access treatment and support. Find tips on caring for yourself, and guidance for friends and family.  What is depression? Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how y

Token indices sequence length is longer than the specified maximum sequence length for this model (1277 > 512). Running this sequence through the model will result in indexing errors


LLM Response
It can be a mental health problem that involves having a low mood or losing interest and enjoyment in things.
Collection name: nhs_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: Can you tell me about depression?

Context: Overview - Depression in adults Depression is more than simply feeling unhappy or fed up for a few days. Most people go through periods of feeling down, but when you're depressed you feel persistently sad for weeks or months, rather than just a few days. Some people think depression is trivial and not a genuine health condition. They're wrong – it is a real illness with real symptoms. Depression is not a sign of weakness or something you can "snap out of" by "pulling yourself together". The good news is that with the right treatment and support, most people with depression can make a full recovery. These pages are about depression in adults. Read about depression 

Token indices sequence length is longer than the specified maximum sequence length for this model (1261 > 512). Running this sequence through the model will result in indexing errors


LLM Response
Depression in adults
Collection name: mind_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: What lifestyle changes are suggested for depression?

Context: might make it hard to understand that these experiences are also symptoms of your depression. And they can be frightening or upsetting, so it's important to seek treatment and support. You might worry that experiencing psychotic symptoms could mean you’re given a diagnosis that doesn’t feel right for you. But discussing all of your symptoms with your doctor can help you get the right support and treatment. How might depression affect my day-to-day life? If you have depression, you might find that it interferes with different aspects of your life. For example, it might affect your ability to work, your relationships, or managing your finances. This can add extra stress to an already difficult experience. It may also feel hard to exp

...

LLM Response
exercise and guided self-help. Talking therapies, such as cognitive behavioural therapy (CBT) , may also be used for mild depression. For moderate to severe depression, a combination of talking therapy and antidepressants is often recommended.
Collection name: mind_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: Are there different types of depressions?

Context: our daily life. Or if they last for several weeks or months.  Are there different types of depression? If you're diagnosed with depression, you might also be told that it is ‘less severe’ or ‘more severe’. This describes how your symptoms are affecting you, and what treatment you're likely to be offered. You may find that the severity of your depression changes over time.  Sometimes you might hear depression being called ‘major depressive disorder’. There are some other types of depression too:  Persistent depressive disord

Token indices sequence length is longer than the specified maximum sequence length for this model (1035 > 512). Running this sequence through the model will result in indexing errors


LLM Response
Are there different types of depression?
Collection name: nhs_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: Are there different types of depressions?

Context: a loss, while depression is an illness. People who are grieving may have symptoms of depression, but they usually improve with time. For some people, bereavement can lead to depression. If you're grieving it's normal to have feelings of sadness. Some other symptoms of depression are rare with grief and bereavement, such as having suicidal thoughts, symptoms of psychosis and feeling hopeless or guilt-ridden. Other types of depression There are different types of depression, and some conditions where depression may be one of the symptoms. These include: postnatal depression – sometimes new mothers, fathers or partners develop depression after they have a baby; this is known as postnatal depression and it's treated in a simila

Token indices sequence length is longer than the specified maximum sequence length for this model (1297 > 512). Running this sequence through the model will result in indexing errors


LLM Response
There are different types of depression, and some conditions where depression may be one of the symptoms. These include: postnatal depression – sometimes new mothers, fathers or partners develop depression after they have a baby; this is known as postnatal depression and it's treated in a similar way to other types of depression, with talking therapies and antidepressant medicines bipolar disorder – also known as "manic depression", in bipolar disorder there are spells of both depression and excessively high mood (mania); the depression symptoms are similar to depression, but the bouts of mania can include harmful behaviour, such as gambling, going on spending sprees and having unsafe sex seasonal affective disorder (SAD) – also known as "winter depression", SAD is a severe type of premenstrual syndrome (PMS) where you have symptoms including feelings of stress, anxiety or low mood during difficult times.


In [43]:
results_df

,mind_response,nhs_response,question
0,It can be a mental health problem that involves having a low mood or losing interest and enjoyment in things.,Depression in adults,Can you tell me about depression?
1,Cut yourself off from other people,"exercise and guided self-help. Talking therapies, such as cognitive behavioural therapy (CBT) , may also be used for mild depression. For moderate to severe depression, a combination of talking therapy and antidepressants is often recommended.",What lifestyle changes are suggested for depression?
2,Are there different types of depression?,"There are different types of depression, and some conditions where depression may be one of the symptoms. These include: postnatal depression – sometimes new mothers, fathers or partners develop depression after they have a baby; this is known as postnatal depression and it's treated in a similar way to other types of depression, with talking therapies and antidepressant medicines bipolar disorder – also known as ""manic depression"", in bipolar disorder there are spells of both depression and excessively high mood (mania); the depression symptoms are similar to depression, but the bouts of mania can include harmful behaviour, such as gambling, going on spending sprees and having unsafe sex seasonal affective disorder (SAD) – also known as ""winter depression"", SAD is a severe type of pre...",Are there different types of depressions?


### chunk_size = 2000 and chunk_overlap = 300

In [44]:
results_df = get_llm_prompt_response(QUESTIONS, enable_chunking = True, is_new_experiment = True, chunk_size = 2000, chunk_overlap = 300)

Found 2 collections
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
Inserting data into collection nhs_collection
Chunking enabled = True
Using chunk_size=2000 and chunk_overlap=300
Inserting data into collection mind_collection
Chunking enabled = True
Using chunk_size=2000 and chunk_overlap=300
Time taken to insert data in chroma store 4.14658222199796 sec
Collection name: mind_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: Can you tell me about depression?

Context: Depression Learn about depression, its symptoms and possible causes, and how you can access treatment and support. Find tips on caring for yourself, and guidance for friends and family.  What is depression? Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you

Token indices sequence length is longer than the specified maximum sequence length for this model (1494 > 512). Running this sequence through the model will result in indexing errors


LLM Response
It can affect your daily life.
Collection name: nhs_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: Can you tell me about depression?

Context: Overview - Depression in adults Depression is more than simply feeling unhappy or fed up for a few days. Most people go through periods of feeling down, but when you're depressed you feel persistently sad for weeks or months, rather than just a few days. Some people think depression is trivial and not a genuine health condition. They're wrong – it is a real illness with real symptoms. Depression is not a sign of weakness or something you can "snap out of" by "pulling yourself together". The good news is that with the right treatment and support, most people with depression can make a full recovery. These pages are about depression in adults. Read about depression in children and young people. How to tell if you have depression Depression aff

Token indices sequence length is longer than the specified maximum sequence length for this model (1692 > 512). Running this sequence through the model will result in indexing errors


LLM Response
Depression is more than simply feeling unhappy or fed up for a few days. Most people go through periods of feeling down, but when you're depressed you feel persistently sad for weeks or months, rather than just a few days. Some people think depression is trivial and not a genuine health condition. They're wrong – it is a real illness with real symptoms. Depression is not a sign of weakness or something you can "snap out of" by "pulling yourself together". The good news is that with the right treatment and support, most people with depression can make a full recovery.
Collection name: mind_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: What lifestyle changes are suggested for depression?

Context: Depression Learn about depression, its symptoms and possible causes, and how you can access treatment and support. Find tips on caring for yourself, and guidance for friends and family.  W

...

LLM Response
getting more exercise and cutting down on alcohol
Collection name: mind_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: Are there different types of depressions?

Context: Depression Learn about depression, its symptoms and possible causes, and how you can access treatment and support. Find tips on caring for yourself, and guidance for friends and family.  What is depression? Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you feel or behave.  The symptoms you experience may vary. How intense they are, how long they last, and how much they affect your daily life can also vary.  If you experience milder depression, you might have low mood but still be able to carry on with your daily life. But things may feel harder and less worthwhile.  If you have more severe depressi

Token indices sequence length is longer than the specified maximum sequence length for this model (1188 > 512). Running this sequence through the model will result in indexing errors


LLM Response
There are some other types of depression too.
Collection name: nhs_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: Are there different types of depressions?

Context: or family member to suggest something is wrong. Doctors describe depression in adults as either less severe (mild) or more severe (moderate or severe), based on: the symptoms, including how often you get symptoms and how bad they are how long depression lasts the impact on your daily life A few people with severe depression may have symptoms of psychosis . You'll usually be asked to answer a set of questions to help doctors assess how severe your condition is. Grief and depression It can be difficult to distinguish between grief and depression. They share many of the same characteristics, but there are important differences between them. Grief is an entirely natural response to a loss, while depression is an illness. P

Token indices sequence length is longer than the specified maximum sequence length for this model (1693 > 512). Running this sequence through the model will result in indexing errors


LLM Response
There are different types of depression, and some conditions where depression may be one of the symptoms. These include: postnatal depression – sometimes new mothers, fathers or partners develop depression after they have a baby; this is known as postnatal depression and it's treated in a similar way to other types of depression, with talking therapies and antidepressant medicines bipolar disorder – also known as "manic depression", in bipolar disorder there are spells of both depression and excessively high mood (mania); the depression symptoms are similar to depression, but the bouts of mania can include harmful behaviour, such as gambling, going on spending sprees and having unsafe sex seasonal affective disorder (SAD) is also known as "winter depression", SAD is a type of depression with a seasonal pattern usually related to winter premenstrual dysphoric


In [45]:
results_df

,mind_response,nhs_response,question
0,It can affect your daily life.,"Depression is more than simply feeling unhappy or fed up for a few days. Most people go through periods of feeling down, but when you're depressed you feel persistently sad for weeks or months, rather than just a few days. Some people think depression is trivial and not a genuine health condition. They're wrong – it is a real illness with real symptoms. Depression is not a sign of weakness or something you can ""snap out of"" by ""pulling yourself together"". The good news is that with the right treatment and support, most people with depression can make a full recovery.",Can you tell me about depression?
1,"loss of interest and enjoyment in things, you might be suffering from depression.",getting more exercise and cutting down on alcohol,What lifestyle changes are suggested for depression?
2,There are some other types of depression too.,"There are different types of depression, and some conditions where depression may be one of the symptoms. These include: postnatal depression – sometimes new mothers, fathers or partners develop depression after they have a baby; this is known as postnatal depression and it's treated in a similar way to other types of depression, with talking therapies and antidepressant medicines bipolar disorder – also known as ""manic depression"", in bipolar disorder there are spells of both depression and excessively high mood (mania); the depression symptoms are similar to depression, but the bouts of mania can include harmful behaviour, such as gambling, going on spending sprees and having unsafe sex seasonal affective disorder (SAD) is also known as ""winter depression"", SAD is a type of depressio...",Are there different types of depressions?


### chunk_size = 1000 and chunk_overlap = 100

In [46]:
results_df = get_llm_prompt_response(QUESTIONS, enable_chunking = True, is_new_experiment = True, chunk_size = 1000, chunk_overlap = 100)

Found 2 collections
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
Inserting data into collection nhs_collection
Chunking enabled = True
Using chunk_size=1000 and chunk_overlap=100
Inserting data into collection mind_collection
Chunking enabled = True
Using chunk_size=1000 and chunk_overlap=100
Time taken to insert data in chroma store 3.1229671469991445 sec
Collection name: mind_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: Can you tell me about depression?

Context: Depression Learn about depression, its symptoms and possible causes, and how you can access treatment and support. Find tips on caring for yourself, and guidance for friends and family.  What is depression? Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how y

Token indices sequence length is longer than the specified maximum sequence length for this model (878 > 512). Running this sequence through the model will result in indexing errors


LLM Response
It starts as sadness then I feel myself shutting down, becoming less capable of coping. Eventually, I just feel numb and empty.
Collection name: nhs_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: Can you tell me about depression?

Context: Overview - Depression in adults Depression is more than simply feeling unhappy or fed up for a few days. Most people go through periods of feeling down, but when you're depressed you feel persistently sad for weeks or months, rather than just a few days. Some people think depression is trivial and not a genuine health condition. They're wrong – it is a real illness with real symptoms. Depression is not a sign of weakness or something you can "snap out of" by "pulling yourself together". The good news is that with the right treatment and support, most people with depression can make a full recovery. These pages are about depression in adults. Read

Token indices sequence length is longer than the specified maximum sequence length for this model (849 > 512). Running this sequence through the model will result in indexing errors


LLM Response
depression is more than simply feeling unhappy or fed up for a few days. Most people go through periods of feeling down, but when you're depressed you feel persistently sad for weeks or months, rather than just a few days.
Collection name: mind_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: What lifestyle changes are suggested for depression?

Context: you. But discussing all of your symptoms with your doctor can help you get the right support and treatment. How might depression affect my day-to-day life? If you have depression, you might find that it interferes with different aspects of your life. For example, it might affect your ability to work, your relationships, or managing your finances. This can add extra stress to an already difficult experience. It may also feel hard to explain your thoughts and feelings to others. You might want to cut yourself off from other people inst

...

LLM Response
Life-changing events, such as bereavement, losing your job or giving birth, can bring it on. People with a family history of depression are more likely to experience it themselves. But you can also become depressed for no obvious reason.
Collection name: mind_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: Are there different types of depressions?

Context: of coping. Eventually, I just feel numb and empty.  When does low mood become depression? We all have times when our mood is low, and we feel sad or fed up. Often these feelings happen for a reason and pass on their own.  But it might be depression if the feelings become so bad that they interfere with our daily life. Or if they last for several weeks or months.  Are there different types of depression? If you're diagnosed with depression, you might also be told that it is ‘less severe’ or ‘more severe’. This describes how your s

Token indices sequence length is longer than the specified maximum sequence length for this model (827 > 512). Running this sequence through the model will result in indexing errors


LLM Response
depression
Collection name: nhs_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: Are there different types of depressions?

Context: and bereavement, such as having suicidal thoughts, symptoms of psychosis and feeling hopeless or guilt-ridden. Other types of depression There are different types of depression, and some conditions where depression may be one of the symptoms. These include: postnatal depression – sometimes new mothers, fathers or partners develop depression after they have a baby; this is known as postnatal depression and it's treated in a similar way to other types of depression, with talking therapies and antidepressant medicines bipolar disorder – also known as "manic depression", in bipolar disorder there are spells of both depression and excessively high mood (mania); the depression symptoms are similar to depression, but the bouts of mania can include harmful beha

Token indices sequence length is longer than the specified maximum sequence length for this model (881 > 512). Running this sequence through the model will result in indexing errors


LLM Response
There are different types of depression, and some conditions where depression may be one of the symptoms.


In [47]:
results_df

,mind_response,nhs_response,question
0,"It starts as sadness then I feel myself shutting down, becoming less capable of coping. Eventually, I just feel numb and empty.","depression is more than simply feeling unhappy or fed up for a few days. Most people go through periods of feeling down, but when you're depressed you feel persistently sad for weeks or months, rather than just a few days.",Can you tell me about depression?
1,Cut yourself off from other people,"Life-changing events, such as bereavement, losing your job or giving birth, can bring it on. People with a family history of depression are more likely to experience it themselves. But you can also become depressed for no obvious reason.",What lifestyle changes are suggested for depression?
2,depression,"There are different types of depression, and some conditions where depression may be one of the symptoms.",Are there different types of depressions?


### chunk_size = 1500 and chunk_overlap = 100

In [48]:
results_df = get_llm_prompt_response(QUESTIONS, enable_chunking = True, is_new_experiment = True, chunk_size = 1500, chunk_overlap = 100)

Found 2 collections
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
Inserting data into collection nhs_collection
Chunking enabled = True
Using chunk_size=1500 and chunk_overlap=100
Inserting data into collection mind_collection
Chunking enabled = True
Using chunk_size=1500 and chunk_overlap=100
Time taken to insert data in chroma store 2.863915079004073 sec
Collection name: mind_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: Can you tell me about depression?

Context: Depression Learn about depression, its symptoms and possible causes, and how you can access treatment and support. Find tips on caring for yourself, and guidance for friends and family.  What is depression? Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how yo

Token indices sequence length is longer than the specified maximum sequence length for this model (1186 > 512). Running this sequence through the model will result in indexing errors


LLM Response
It can affect your daily life.
Collection name: nhs_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: Can you tell me about depression?

Context: Overview - Depression in adults Depression is more than simply feeling unhappy or fed up for a few days. Most people go through periods of feeling down, but when you're depressed you feel persistently sad for weeks or months, rather than just a few days. Some people think depression is trivial and not a genuine health condition. They're wrong – it is a real illness with real symptoms. Depression is not a sign of weakness or something you can "snap out of" by "pulling yourself together". The good news is that with the right treatment and support, most people with depression can make a full recovery. These pages are about depression in adults. Read about depression in children and young people. How to tell if you have depression Depression aff

Token indices sequence length is longer than the specified maximum sequence length for this model (1262 > 512). Running this sequence through the model will result in indexing errors


LLM Response
Depression is more than simply feeling unhappy or fed up for a few days. Most people go through periods of feeling down, but when you're depressed you feel persistently sad for weeks or months, rather than just a few days.
Collection name: mind_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: What lifestyle changes are suggested for depression?

Context: treatment. How might depression affect my day-to-day life? If you have depression, you might find that it interferes with different aspects of your life. For example, it might affect your ability to work, your relationships, or managing your finances. This can add extra stress to an already difficult experience. It may also feel hard to explain your thoughts and feelings to others. You might want to cut yourself off from other people instead. This could lead to you feeling isolated and lonely. It feels like I'm stuck under a huge gre

...

LLM Response
getting more exercise , cutting down on alcohol , giving up smoking and eating healthily
Collection name: mind_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: Are there different types of depressions?

Context: severe’. This describes how your symptoms are affecting you, and what treatment you're likely to be offered. You may find that the severity of your depression changes over time.  Sometimes you might hear depression being called ‘major depressive disorder’. There are some other types of depression too:  Persistent depressive disorder (PDD). PDD is continuous depression that lasts for two years or more. You may also hear it called dysthymia or chronic depression. Seasonal affective disorder (SAD). SAD is depression that occurs at a particular time of year, or during a particular season. See our page on SAD for more information. Antenatal depression. This is depression that occu

Token indices sequence length is longer than the specified maximum sequence length for this model (1187 > 512). Running this sequence through the model will result in indexing errors


LLM Response
There are some other types too: Persistent depressive disorder (PDD). PDD is continuous depression that lasts for two years or more. You may also hear it called ‘Dysthymia or chronic depression.
Collection name: nhs_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: Are there different types of depressions?

Context: Other types of depression There are different types of depression, and some conditions where depression may be one of the symptoms. These include: postnatal depression – sometimes new mothers, fathers or partners develop depression after they have a baby; this is known as postnatal depression and it's treated in a similar way to other types of depression, with talking therapies and antidepressant medicines bipolar disorder – also known as "manic depression", in bipolar disorder there are spells of both depression and excessively high mood (mania); the depression symptoms a

Token indices sequence length is longer than the specified maximum sequence length for this model (1236 > 512). Running this sequence through the model will result in indexing errors


LLM Response
There are different types of depression, and some conditions where depression may be one of the symptoms. These include: postnatal depression – sometimes new mothers, fathers or partners develop depression after they have a baby; this is known as postnatal depression and it's treated in a similar way to other types of depression, with talking therapies and antidepressant medicines bipolar disorder – also known as "manic depression", in bipolar disorder there are spells of both depression and excessively high mood (mania); the depression symptoms are similar to depression, but the bouts of mania can include harmful behaviour, such as gambling, going on spending sprees and having unsafe sex seasonal affective disorder (SAD) – also known as "winter depression", SAD is a severe type of premenstrual syndrome (PMS) where you have symptoms including feelings of depression and anxiety in the weeks before your period. When to seek


In [49]:
results_df

,mind_response,nhs_response,question
0,It can affect your daily life.,"Depression is more than simply feeling unhappy or fed up for a few days. Most people go through periods of feeling down, but when you're depressed you feel persistently sad for weeks or months, rather than just a few days.",Can you tell me about depression?
1,Cut yourself off from other people,"getting more exercise , cutting down on alcohol , giving up smoking and eating healthily",What lifestyle changes are suggested for depression?
2,There are some other types too: Persistent depressive disorder (PDD). PDD is continuous depression that lasts for two years or more. You may also hear it called ‘Dysthymia or chronic depression.,"There are different types of depression, and some conditions where depression may be one of the symptoms. These include: postnatal depression – sometimes new mothers, fathers or partners develop depression after they have a baby; this is known as postnatal depression and it's treated in a similar way to other types of depression, with talking therapies and antidepressant medicines bipolar disorder – also known as ""manic depression"", in bipolar disorder there are spells of both depression and excessively high mood (mania); the depression symptoms are similar to depression, but the bouts of mania can include harmful behaviour, such as gambling, going on spending sprees and having unsafe sex seasonal affective disorder (SAD) – also known as ""winter depression"", SAD is a severe type of pre...",Are there different types of depressions?


### chunk_size = 2000 and chunk_overlap = 100

In [51]:
results_df = get_llm_prompt_response(QUESTIONS, enable_chunking = True, is_new_experiment = True, chunk_size = 2000, chunk_overlap = 100)

Found 2 collections
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
Inserting data into collection nhs_collection
Chunking enabled = True
Using chunk_size=2000 and chunk_overlap=100
Inserting data into collection mind_collection
Chunking enabled = True
Using chunk_size=2000 and chunk_overlap=100
Time taken to insert data in chroma store 3.924305820000882 sec
Collection name: mind_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: Can you tell me about depression?

Context: Depression Learn about depression, its symptoms and possible causes, and how you can access treatment and support. Find tips on caring for yourself, and guidance for friends and family.  What is depression? Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how yo

Token indices sequence length is longer than the specified maximum sequence length for this model (1356 > 512). Running this sequence through the model will result in indexing errors


LLM Response
It's a mental health problem that involves having a low mood or losing interest and enjoyment in things.
Collection name: nhs_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: Can you tell me about depression?

Context: Overview - Depression in adults Depression is more than simply feeling unhappy or fed up for a few days. Most people go through periods of feeling down, but when you're depressed you feel persistently sad for weeks or months, rather than just a few days. Some people think depression is trivial and not a genuine health condition. They're wrong – it is a real illness with real symptoms. Depression is not a sign of weakness or something you can "snap out of" by "pulling yourself together". The good news is that with the right treatment and support, most people with depression can make a full recovery. These pages are about depression in adults. Read about depression in ch

Token indices sequence length is longer than the specified maximum sequence length for this model (1650 > 512). Running this sequence through the model will result in indexing errors


LLM Response
Depression is more than simply feeling unhappy or fed up for a few days. Most people go through periods of feeling down, but when you're depressed you feel persistently sad for weeks or months, rather than just a few days. Some people think depression is trivial and not a genuine health condition. They're wrong – it is a real illness with real symptoms. Depression is not a sign of weakness or something you can "snap out of" by "pulling yourself together".
Collection name: mind_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: What lifestyle changes are suggested for depression?

Context: health. And your doctor might offer you treatment for depression without realising that you also experience other symptoms. If you think you're experiencing other symptoms, you can talk to your doctor about this to make sure you're getting the right treatment. Depression Learn about depression, its sy

...

LLM Response
Get more exercise and cut down on alcohol
Collection name: mind_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: Are there different types of depressions?

Context: Depression Learn about depression, its symptoms and possible causes, and how you can access treatment and support. Find tips on caring for yourself, and guidance for friends and family.  What is depression? Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you feel or behave.  The symptoms you experience may vary. How intense they are, how long they last, and how much they affect your daily life can also vary.  If you experience milder depression, you might have low mood but still be able to carry on with your daily life. But things may feel harder and less worthwhile.  If you have more severe depression, you 

Token indices sequence length is longer than the specified maximum sequence length for this model (1055 > 512). Running this sequence through the model will result in indexing errors


LLM Response
Persistent depressive disorder
Collection name: nhs_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: Are there different types of depressions?

Context: you get symptoms and how bad they are how long depression lasts the impact on your daily life A few people with severe depression may have symptoms of psychosis . You'll usually be asked to answer a set of questions to help doctors assess how severe your condition is. Grief and depression It can be difficult to distinguish between grief and depression. They share many of the same characteristics, but there are important differences between them. Grief is an entirely natural response to a loss, while depression is an illness. People who are grieving may have symptoms of depression, but they usually improve with time. For some people, bereavement can lead to depression. If you're grieving it's normal to have feelings of sadness. Some o

Token indices sequence length is longer than the specified maximum sequence length for this model (1651 > 512). Running this sequence through the model will result in indexing errors


LLM Response
There are different types of depression, and some conditions where depression may be one of the symptoms. These include: postnatal depression – sometimes new mothers, fathers or partners develop depression after they have a baby; this is known as postnatal depression and it's treated in a similar way to other types of depression, with talking therapies and antidepressant medicines bipolar disorder – also known as "manic depression", in bipolar disorder there are spells of both depression and excessively high mood (mania); the depression symptoms are similar to depression, but the bouts of mania can include harmful behaviour, such as gambling, going on spending sprees and having unsafe sex seasonal affective disorder (SAD) – also known as "winter depression", SAD is a severe type of premenstrual syndrome (PMS) where you have symptoms including feelings of stress, anxiety or low mood during difficult times.


In [52]:
results_df

,mind_response,nhs_response,question
0,It's a mental health problem that involves having a low mood or losing interest and enjoyment in things.,"Depression is more than simply feeling unhappy or fed up for a few days. Most people go through periods of feeling down, but when you're depressed you feel persistently sad for weeks or months, rather than just a few days. Some people think depression is trivial and not a genuine health condition. They're wrong – it is a real illness with real symptoms. Depression is not a sign of weakness or something you can ""snap out of"" by ""pulling yourself together"".",Can you tell me about depression?
1,"other symptoms. If you think you're experiencing other symptoms, you can talk to your doctor about this to make sure you're getting the right treatment.",Get more exercise and cut down on alcohol,What lifestyle changes are suggested for depression?
2,Persistent depressive disorder,"There are different types of depression, and some conditions where depression may be one of the symptoms. These include: postnatal depression – sometimes new mothers, fathers or partners develop depression after they have a baby; this is known as postnatal depression and it's treated in a similar way to other types of depression, with talking therapies and antidepressant medicines bipolar disorder – also known as ""manic depression"", in bipolar disorder there are spells of both depression and excessively high mood (mania); the depression symptoms are similar to depression, but the bouts of mania can include harmful behaviour, such as gambling, going on spending sprees and having unsafe sex seasonal affective disorder (SAD) – also known as ""winter depression"", SAD is a severe type of pre...",Are there different types of depressions?


In [12]:
results_df = get_llm_prompt_response(QUESTIONS, enable_chunking = True, is_new_experiment = True, chunk_size = 700, chunk_overlap = 50)

Found 2 collections
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
Inserting data into collection nhs_collection
Chunking enabled = True
Using chunk_size=700 and chunk_overlap=50
Inserting data into collection mind_collection
Chunking enabled = True
Using chunk_size=700 and chunk_overlap=50
Time taken to insert data in chroma store 2.5272354089993314 sec
Collection name: mind_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: Can you tell me about depression?

Context: Depression Learn about depression, its symptoms and possible causes, and how you can access treatment and support. Find tips on caring for yourself, and guidance for friends and family.  What is depression? Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you f

Token indices sequence length is longer than the specified maximum sequence length for this model (608 > 512). Running this sequence through the model will result in indexing errors


LLM Response
It's dark and isolating, smothering me at every opportunity.
Collection name: nhs_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: Can you tell me about depression?

Context: Read about depression in children and young people. How to tell if you have depression Depression affects people in different ways and can cause a wide variety of symptoms. They range from lasting feelings of unhappiness and hopelessness, to losing interest in the things you used to enjoy and feeling very tearful. Many people with depression also have symptoms of anxiety . There can be physical symptoms too, such as feeling constantly tired, sleeping badly, having no appetite or sex drive, and various aches and pains. The symptoms of depression range from mild to severe. At its mildest, you may simply feel persistently low in spirit, while severe depression can make you feel suicidal, that of others having no mo

Token indices sequence length is longer than the specified maximum sequence length for this model (597 > 512). Running this sequence through the model will result in indexing errors


LLM Response
Depression affects people in different ways and can cause a wide variety of symptoms. They range from lasting feelings of unhappiness and hopelessness, to losing interest in the things you used to enjoy and feeling very tearful.
Collection name: mind_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: What lifestyle changes are suggested for depression?

Context: Losing interest in sex Difficulty remembering or concentrating on things Using more tobacco, alcohol or other drugs than usual Difficulty sleeping, or sleeping too much No appetite and losing weight, or eating more than usual and gaining weight Physical aches and pains with no obvious physical cause Moving very slowly, or being restless and agitated It felt like I was really tired, all the time. I had no energy or emotion about anything. Anxiety It's very common to experience depression and anxiety together. Some symptoms of de

...

LLM Response
getting more exercise , cutting down on alcohol , giving up smoking and eating healthily
Collection name: mind_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: Are there different types of depressions?

Context: If you're diagnosed with depression, you might also be told that it is ‘less severe’ or ‘more severe’. This describes how your symptoms are affecting you, and what treatment you're likely to be offered. You may find that the severity of your depression changes over time.  Sometimes you might hear depression being called ‘major depressive disorder’. There are some other types of depression too:  Persistent depressive disorder (PDD). PDD is continuous depression that lasts for two years or more. You may also hear it called dysthymia or chronic depression. Seasonal affective disorder (SAD). SAD is depression that occurs at a particular time of year, or during a particular season

Token indices sequence length is longer than the specified maximum sequence length for this model (604 > 512). Running this sequence through the model will result in indexing errors


LLM Response
There are some other types of depression too: Persistent depressive disorder (PDD). PDD is continuous depression that lasts for two years or more. You may also hear it called dysthymia or chronic depression.
Collection name: nhs_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: Are there different types of depressions?

Context: to have feelings of sadness. Some other symptoms of depression are rare with grief and bereavement, such as having suicidal thoughts, symptoms of psychosis and feeling hopeless or guilt-ridden. Other types of depression There are different types of depression, and some conditions where depression may be one of the symptoms. These include: postnatal depression – sometimes new mothers, fathers or partners develop depression after they have a baby; this is known as postnatal depression and it's treated in a similar way to other types of depression, with talking t

Token indices sequence length is longer than the specified maximum sequence length for this model (612 > 512). Running this sequence through the model will result in indexing errors


LLM Response
There are different types of depression, and some conditions where depression may be one of the symptoms. These include:


In [13]:
results_df

,mind_response,nhs_response,question
0,"It's dark and isolating, smothering me at every opportunity.","Depression affects people in different ways and can cause a wide variety of symptoms. They range from lasting feelings of unhappiness and hopelessness, to losing interest in the things you used to enjoy and feeling very tearful.",Can you tell me about depression?
1,"Using more tobacco, alcohol or other drugs than usual Difficulty sleeping, or sleeping too much No appetite and losing weight, or eating more than usual and gaining weight Physical aches and pains with no obvious physical cause Moving very slowly, or being restless and agitated It felt like I was really tired, all the time. I had no energy or emotion about anything. Anxiety It's very common to experience depression and anxiety together.","getting more exercise , cutting down on alcohol , giving up smoking and eating healthily",What lifestyle changes are suggested for depression?
2,There are some other types of depression too: Persistent depressive disorder (PDD). PDD is continuous depression that lasts for two years or more. You may also hear it called dysthymia or chronic depression.,"There are different types of depression, and some conditions where depression may be one of the symptoms. These include:",Are there different types of depressions?


In [31]:
results_df = get_llm_prompt_response(QUESTIONS, enable_chunking = True, is_new_experiment = True, chunk_size = 780, chunk_overlap = 50)

Found 2 collections
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
Inserting data into collection nhs_collection
Chunking enabled = True
Using chunk_size=780 and chunk_overlap=50
Inserting data into collection mind_collection
Chunking enabled = True
Using chunk_size=780 and chunk_overlap=50
Time taken to insert data in chroma store 2.3054386250005336 sec
Collection name: mind_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: Can you tell me about depression?

Context: with different aspects of your life. For example, it might affect your ability to work, your relationships, or managing your finances. This can add extra stress to an already difficult experience. It may also feel hard to explain your thoughts and feelings to others. You might want to cut yourself off from other people instead. This could lead to you feeling isolated and lo

Token indices sequence length is longer than the specified maximum sequence length for this model (678 > 512). Running this sequence through the model will result in indexing errors


LLM Response
It can also cause other mental health problems, such as: Bipolar disorder, Borderline personality disorder (BPD) and other personality disorders, Schizoaffective disorder.
Collection name: nhs_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: Can you tell me about depression?

Context: Depression affects people in different ways and can cause a wide variety of symptoms. They range from lasting feelings of unhappiness and hopelessness, to losing interest in the things you used to enjoy and feeling very tearful. Many people with depression also have symptoms of anxiety . There can be physical symptoms too, such as feeling constantly tired, sleeping badly, having no appetite or sex drive, and various aches and pains. The symptoms of depression range from mild to severe. At its mildest, you may simply feel persistently low in spirit, while severe depression can make you feel suicidal, tha

Token indices sequence length is longer than the specified maximum sequence length for this model (663 > 512). Running this sequence through the model will result in indexing errors


LLM Response
depression and grief
Collection name: mind_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: What lifestyle changes are suggested for depression?

Context: with different aspects of your life. For example, it might affect your ability to work, your relationships, or managing your finances. This can add extra stress to an already difficult experience. It may also feel hard to explain your thoughts and feelings to others. You might want to cut yourself off from other people instead. This could lead to you feeling isolated and lonely. It feels like I'm stuck under a huge grey-black cloud. It's dark and isolating, smothering me at every opportunity. Can depression be a symptom of other mental health problems? Symptoms of depression can also be part of other mental health problems, such as: Bipolar disorder, Borderline personality disorder (BPD) and other personality disorders, Schizoaffec

...

LLM Response
getting more exercise , cutting down on alcohol , giving up smoking and eating healthily
Collection name: mind_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: Are there different types of depressions?

Context: and what treatment you're likely to be offered. You may find that the severity of your depression changes over time.  Sometimes you might hear depression being called ‘major depressive disorder’. There are some other types of depression too:  Persistent depressive disorder (PDD). PDD is continuous depression that lasts for two years or more. You may also hear it called dysthymia or chronic depression. Seasonal affective disorder (SAD). SAD is depression that occurs at a particular time of year, or during a particular season. See our page on SAD for more information. Antenatal depression. This is depression that occurs while you are pregnant. It is sometimes called prenatal de

Token indices sequence length is longer than the specified maximum sequence length for this model (689 > 512). Running this sequence through the model will result in indexing errors


LLM Response
There are some other types of depression too: Persistent depressive disorder (PDD). PDD is continuous depression that lasts for two years or more. You may also hear it called dysthymia or chronic depression.
Collection name: nhs_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: Are there different types of depressions?

Context: It can be difficult to distinguish between grief and depression. They share many of the same characteristics, but there are important differences between them. Grief is an entirely natural response to a loss, while depression is an illness. People who are grieving may have symptoms of depression, but they usually improve with time. For some people, bereavement can lead to depression. If you're grieving it's normal to have feelings of sadness. Some other symptoms of depression are rare with grief and bereavement, such as having suicidal thoughts, symptoms of ps

Token indices sequence length is longer than the specified maximum sequence length for this model (669 > 512). Running this sequence through the model will result in indexing errors


LLM Response
Grief


In [32]:
results_df

,mind_response,nhs_response,question
0,"It can also cause other mental health problems, such as: Bipolar disorder, Borderline personality disorder (BPD) and other personality disorders, Schizoaffective disorder.",depression and grief,Can you tell me about depression?
1,cut yourself off from other people,"getting more exercise , cutting down on alcohol , giving up smoking and eating healthily",What lifestyle changes are suggested for depression?
2,There are some other types of depression too: Persistent depressive disorder (PDD). PDD is continuous depression that lasts for two years or more. You may also hear it called dysthymia or chronic depression.,Grief,Are there different types of depressions?


In [24]:
results_df = get_llm_prompt_response(QUESTIONS, enable_chunking = True, is_new_experiment = True, chunk_size = 500, chunk_overlap = 50)

Found 2 collections
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
Inserting data into collection nhs_collection
Chunking enabled = True
Using chunk_size=500 and chunk_overlap=50
Inserting data into collection mind_collection
Chunking enabled = True
Using chunk_size=500 and chunk_overlap=50
Time taken to insert data in chroma store 2.406398352000906 sec
Collection name: mind_collection
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512
LLM prompt
Question: Can you tell me about depression?

Context: Depression Learn about depression, its symptoms and possible causes, and how you can access treatment and support. Find tips on caring for yourself, and guidance for friends and family.  What is depression? Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you fe

In [25]:
results_df

,mind_response,nhs_response,question
0,"It can interfere with your ability to work, your relationships, or managing your finances.",The psychological symptoms of depression include: continuous low mood or sadness feeling hopeless and helpless having low self-esteem feeling tearful feeling guilt-ridden feeling irritable and intolerant of others having no motivation or interest in things finding it difficult to make decisions not getting any enjoyment out of life feeling anxious or worried having suicidal thoughts or thoughts of harming yourself,Can you tell me about depression?
1,Cut yourself off from other people instead,"getting more exercise , cutting down on alcohol , giving up smoking and eating healthily",What lifestyle changes are suggested for depression?
2,more severe,"There are different types of depression, and some conditions where depression may be one of the symptoms. These include:",Are there different types of depressions?
